<a href="https://colab.research.google.com/github/Arevik2004/arevik/blob/main/%D0%BB%D0%B0%D0%B1%D0%B0_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Отчет по лабораторной работе на тему:MongoDB
# Выполнила работу студент группы ПИН-Б-З-22-1
# Элбакян Аревик Ашотовна
Задачи
1. Вывести JSON-документ, содержащий { "hello": "world" }.
2. Выбрать город через регистронезависимое регулярное выражение, содержащее слово new.
3. Найти все города, названия которых содержат букву 'e' и известны своей едой или пивом.
4. Создать новую базу данных под названием blogger с коллекцией articles. Вставить новую
статью с именем автора, электронной почтой, датой создания и текстом.
5. Обновить статью массивом комментариев, содержащим комментарий с автором и текстом.
6. Запустить запрос из внешнего файла JavaScript, который вы создадите сами

# 1. Вывести JSON-документ, содержащий { "hello": "world" }.

In [ ]:
#Вывод JSON-документа
printjson({ "hello": "world" })


In [ ]:
#Результат
{ "hello" : "world" }

# 2. Выбрать город через регистронезависимое регулярное выражение, содержащее слово new.

In [ ]:
#Используем регулярное выражение с флагом 'i' для регистронезависимости
db.cities.find({ "city.name": /new/i })
#Флаг i делает поиск нечувствительным к регистру.

# 3. Найти все города, названия которых содержат букву 'e' и известны своей едой или пивом.

In [ ]:
#Сложный запрос с несколькими условиями
db.cities.find({
  $and: [
    { name: /e/i },  // содержит букву 'e' в любом регистре
    {
      $or: [
        { famousFor: "food" },  // известен едой
        { famousFor: "beer" },   // известен пивом
        { cuisine: { $exists: true } },  // или имеет информацию о кухне
        { breweries: { $exists: true } } // или имеет информацию о пивоварнях
      ]
    }
  ]
})

#Альтернативный вариант с оператором
db.cities.find({
  name: { $regex: "e", $options: "i" },
  $or: [
    { tags: "food" },
    { tags: "beer" },
    { attractions: /food|beer|cuisine|brewery/i }
  ]
})

# 4. Создать новую базу данных под названием blogger с коллекцией articles. Вставить новую статью с именем автора, электронной почтой, датой создания и текстом.


In [ ]:
#Создаем и переключаемся на новую базу данных
use blogger

#Создаем коллекцию articles и вставляем документ
db.articles.insertOne({
  author: "Элбакян Аревик",
  email: "arevelbakan@gmail.com",
  createdAt: new Date(),
  text: "Это текст моей первой статьи в блоге...",
  title: "Мой первый пост"
})

In [ ]:
#Результат
{
  acknowledged: true,
  insertedId: ObjectId("650a1b2c3d4e5f6789a0b1c2")
}

# 5. Обновить статью массивом комментариев, содержащим комментарий с автором и текстом.


In [ ]:
#Обновляем статью, добавляя массив комментариев
db.articles.updateOne(
  { _id: ObjectId("650a1b2c3d4e5f6789a0b1c2") }, // ID из предыдущей вставки
  {
    $set: {
      comments: [
        {
          author: "Анна Сидорова",
          text: "Отличная статья! Спасибо за информацию.",
          date: new Date()
        },
        {
          author: "Элбакян Аревик",
          text: "Интересный взгляд на проблему.",
          date: new Date()
        }
      ],
      updatedAt: new Date()
    }
  }
)

In [ ]:
#Результат
{
  acknowledged: true,
  matchedCount: 1,
  modifiedCount: 1
}

# 6. Запустить запрос из внешнего файла JavaScript, который вы создадите сами

In [ ]:
#query.js
print("Запуск внешнего скрипта ");
#1. Подсчет статей по авторам
print("\n1. Количество статей по авторам:");
db.articles.aggregate([
  { $group: { _id: "$author", count: { $sum: 1 } } },
  { $sort: { count: -1 } }
]).forEach(function(doc) {
  print(doc._id + ": " + doc.count + " статей");
});

#2. Поиск статей с комментариями
print("\n2. Статьи с комментариями:");
db.articles.find(
  { comments: { $exists: true, $ne: [] } },
  { title: 1, author: 1, "comments": { $slice: 1 } }
).forEach(function(article) {
  print("Статья: " + article.title);
  print("Автор: " + article.author);
  print("Первый комментарий от: " + article.comments[0].author);
  print("---");
});

#3. Статистика
print("\n3. Общая статистика:");
print("Всего статей: " + db.articles.countDocuments());
print("Статей с комментариями: " + db.articles.countDocuments({ comments: { $exists: true, $ne: [] } }));

# 7. Реализуйте метод finalize, который выводит количество как общее значение.Установите драйвер Mongo для языка по вашему выбору и подключитесь к базе данных.Заполните коллекцию через него и проиндексируйте одно из полей


In [ ]:
#Создаем коллекцию sales
use testdb
db.sales.drop()

db.sales.insertMany([
  { product: "Laptop", category: "Electronics", amount: 1500, quantity: 2 },
  { product: "Mouse", category: "Electronics", amount: 50, quantity: 10 },
  { product: "Chair", category: "Furniture", amount: 200, quantity: 5 },
  { product: "Table", category: "Furniture", amount: 500, quantity: 3 },
  { product: "Monitor", category: "Electronics", amount: 300, quantity: 7 }
])

In [ ]:
#Map-функция: группируем по категории
var mapFunction = function() {
  emit(this.category, {
    count: 1,
    totalAmount: this.amount * this.quantity,
    totalQuantity: this.quantity
  });
};

#Reduce-функция: суммируем значения
var reduceFunction = function(key, values) {
  var result = { count: 0, totalAmount: 0, totalQuantity: 0 };

  values.forEach(function(value) {
    result.count += value.count;
    result.totalAmount += value.totalAmount;
    result.totalQuantity += value.quantity || value.totalQuantity;
  });

  return result;
};

#Finalize-функция: добавляем вычисляемые поля
var finalizeFunction = function(key, reducedValue) {
  reducedValue.totalItems = reducedValue.count;

 #Добавляем средние значения
  reducedValue.averageAmount = reducedValue.totalAmount / reducedValue.totalQuantity;
  reducedValue.averagePerItem = reducedValue.totalAmount / reducedValue.count;

  #Форматируем вывод
  reducedValue.formattedTotal = "$" + reducedValue.totalAmount.toFixed(2);
  reducedValue.description = key + " category has " + reducedValue.totalItems + " items";

  return reducedValue;
};

// Запускаем MapReduce
db.sales.mapReduce(
  mapFunction,
  reduceFunction,
  {
    out: "sales_summary",
    finalize: finalizeFunction,
    query: { amount: { $gt: 0 } } // Опциональный фильтр
  }
);

In [ ]:
#Результат
db.sales_summary.find().sort({_id: 1})


In [ ]:
#Результат в терминале
 [
  {
    "_id": "Electronics",
    "value": {
      "count": 3,
      "totalAmount": 7850,
      "totalQuantity": 19,
      "totalItems": 3,
      "averageAmount": 413.1578947368421,
      "averagePerItem": 2616.6666666666665,
      "formattedTotal": "$7850.00",
      "description": "Electronics category has 3 items"
    }
  },
  {
    "_id": "Furniture",
    "value": {
      "count": 2,
      "totalAmount": 2500,
      "totalQuantity": 8,
      "totalItems": 2,
      "averageAmount": 312.5,
      "averagePerItem": 1250,
      "formattedTotal": "$2500.00",
      "description": "Furniture category has 2 items"
    }
  }
]